## imports

In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import shutil
import re

In [ ]:
IMAGES_DIR = Path(r"C:\Users\ADMIN\Downloads\emotion classification\Data\images_renamed")   # folder with  images
CSV_PATH   = Path(r"C:\Users\ADMIN\Downloads\emotion classification\Data\TIF_labels.xlsx")  # CSV with columns: image,label
OUTPUT_DIR = Path(r"C:\Users\ADMIN\Downloads\emotion classification\Data\data_emotions")   # where splits/CSVs (and optional folders) go

# split ratios
TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.70, 0.15, 0.15  # must sum to 1

# If your CSV image IDs don't include extensions, we’ll search these:
ALLOWED_EXTS = [".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"]

# If original filenames have suffixes (e.g., "...-4317HA") but your CSV has "...-4317",
# set this to True to allow prefix matches when exact matches fail:
ALLOW_PREFIX_MATCH = True

# If True, also copy files into ImageNet-style folders data/{split}/{label}/img.jpg
MAKE_CLASS_FOLDERS = False
# ======================

# Standardize class names (case-insensitive) to your 7 classes
CANONICAL = {
    "happy": "happy",
    "sad": "sad",
    "anger": "anger",
    "angry": "anger",
    "neutral": "neutral",
    "disgust": "disgust",
    "surprise": "surprise",
    "surprised": "surprise",
    "fear": "fear",
    "scared": "fear",
}
